In [ ]:
import pandas as pd
import uuid
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import folium
from folium.plugins import HeatMap


caminho_arquivo = r'C:\Users\User\Desktop\Ada Hackaton\adahack-2024-dados\base_dados\base_de_dados.csv'
df = pd.read_csv(caminho_arquivo, encoding='utf-8')


### Análise da base: correção de caracteres

In [ ]:
#Foi verificado que a grafia das letras está errada tanto nas células quanto no nome das colunas. Vamos ajustar: 
# Dicionário de substituições
substituicoes = {
    'Ã©': 'é', 'Ã¢': 'â', 'Ãº': 'ú', 'Ã­': 'í', 'Ã§': 'ç',
    'Ã¡': 'á', 'Ã³': 'ó', 'Ãª': 'ê', 'Ã®': 'î', 'Ã´': 'ô',
    'Ã»': 'û', 'Ã£': 'ã', 'Ãµ': 'õ', 'Ã ': 'à', 'Ã¨': 'è',
    'Ã¬': 'ì', 'Ã¹': 'ù', 'Ã¤': 'ä', 'Ã«': 'ë', 'Ã¯': 'ï',
    'Ã¶': 'ö', 'Ã¼': 'ü'
}

# Substituir os caracteres nos nomes das colunas
df.columns = [coluna for coluna in df.columns]
for original, correcao in substituicoes.items():
    df.columns = [coluna.replace(original, correcao) for coluna in df.columns]

# Substituir os caracteres na base de dados
df.replace(substituicoes, regex=True, inplace=True)

In [ ]:
display(df)

### Ajustes pontuais: estados

In [ ]:
# Dicionário de mapeamento de estados
mapeamento_estados = {
    'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA',
    'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO',
    'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG',
    'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR', 'Pernambuco': 'PE', 'Piauí': 'PI',
    'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP',
    'Sergipe': 'SE', 'Tocantins': 'TO'
}

# Aplicando o mapeamento de estados
df['Estado'] = df['Estado'].map(mapeamento_estados)

# Mostra o DataFrame com as siglas dos estados
display(df.head())

### Ajustes pontuais: acentuação, espaços

In [ ]:
#As colunas também estão com erros: Vamos manter o padrão de formatação: Minúsculo, espaços sinalizados com sublinhado, e ajustes gráficos das letras
from unidecode import unidecode

# Função para remover caracteres especiais de strings
def remover_acentos(texto):
    if isinstance(texto, str):
        return unidecode(texto)
    else:
        return texto


# Aplicar a função nas colunas
df.columns = [remover_acentos(coluna.replace(' ', '_').lower()) for coluna in df.columns]

# Aplicar a função em todo o DataFrame, apenas em dados do tipo string
df = df.applymap(remover_acentos)

In [ ]:
display(df)

### Ajustes pontuais: colunas sem função e subtituição de ID sequencial por padrão hexadecimal

In [ ]:
# Removendo a coluna "id", "endereco" e "nome"
df.drop(columns=['id', 'id.1', 'nome', 'endereco'], inplace=True)  # Remove as colunas "id" e "id.1"

# Função para gerar IDs únicos de 4 dígitos (letras e números)
def gerar_id():
    return str(uuid.uuid4().hex[:4]).upper()

# Criando uma nova coluna "ID" com IDs únicos de 4 dígitos
df['ID'] = [gerar_id() for _ in range(len(df))]

# Reorganizando as colunas para ter a nova coluna "ID" como a primeira
colunas = df.columns.tolist()
colunas = ['ID'] + [coluna for coluna in colunas if coluna != 'ID']
df = df[colunas]

# Mostra o DataFrame com a nova coluna "ID"
print(df.head())

### Análise de dados: Entendimento dos dados relacionando com o mundo exterior

#### Relacionar dados de idade com força de trabalho, com ênfase no ECA e CLT

In [ ]:
# Convertendo a coluna 'Idade' para números numéricos
df['idade'] = pd.to_numeric(df['idade'], errors='coerce')

# Conta quantas vezes a idade é menor que 14 anos
df[df["idade"] < 14]["idade"].value_counts()

#Seguindo a legislaçãos, os dados do Estatuto da Criança e do Adolescente (ECA) proíbe qualquer trabalho para menores de 14 anos, 
#exceto na condição de aprendiz. A partir dos 14 anos, os jovens podem trabalhar como aprendizes, seguindo as diretrizes e bases da 
#legislação de educação em vigor
menores_trabalhando = df[(df['idade'] < 14)]

# Mostra o DataFrame com as idades corrigidas
display(menores_trabalhando)

In [ ]:
#Veja, na base acima, temos um problema, pessoas menores de 14 anos. 
menores_trabalhando_clt_eca = df[(df['idade'] >= 14) & (df['idade'] <= 17) ]

In [ ]:
menores_trabalhando_clt_eca = df[(df['idade'] >= 14) & (df['idade'] <= 17) & (df['senioridade'] == 'Estagiario')]
menores_trabalhando_clt_eca

#A base acima poderia ser somente aprendiz, salvo a condição de estagiário técnico

In [ ]:
#Vamos tirar esses menores para evitar sanções. Destacando numa base a parte é interessante que a equipe de de RH analise os casos:

#Já vimos quem são os menores, agora vamos condicionar os termos já agrupados. 

menores_irregulares = df[(df['idade'] >= 14) & (df['idade'] <= 17) & ~(df['senioridade'].isin(['Aprendiz', 'Estagiario']))]

#Aqui vamos criar uma base exclusiva de análise
df_menores_irregulares = menores_irregulares.copy()

#Para limpar a base, vamos tirar os dados
df = df.drop(menores_irregulares.index)

df_menores_irregulares.to_csv('menores_trabalhando_irregularmente.csv', index=False)

#### Análise de dados: Menores exercendo funções na empresa

In [ ]:
# Convertendo a coluna 'tempo_de_casa' para números numéricos
df['tempo_de_casa'] = pd.to_numeric(df['tempo_de_casa'], errors='coerce')

# Corrigindo o tempo_de_casa para nulo se for maior que a idade
df.loc[df['tempo_de_casa'] > df['idade'], 'tempo_de_casa'] = pd.NA

# Convertendo tempos de casa para números inteiros
df['tempo_de_casa'] = df['tempo_de_casa'].astype('Int64')

# Contando quantas linhas têm o tempo_de_casa + 14 maior que a idade
condicao = df['tempo_de_casa'] + 14 > df['idade']
total_linhas = condicao.sum()

print(f"Total de linhas com tempo de casa + 14 maior que a Idade: {total_linhas}")

#### Ajustes pontuais: Transformando valores negativos em nulos

In [ ]:
#Transformando valores negativos em nulos
df.loc[df['tempo_de_casa'] < 0, 'tempo_de_casa'] = np.nan

#Transformando valores tempo_de_casa + 14 maior que a idade em nulos
df.loc[df['tempo_de_casa'] + 14 >  df['idade'], 'tempo_de_casa'] = np.nan

#### Ajustes pontuais: Transformando valores vazios em informações

In [ ]:
# Vamos substituir por "Desconhecido - Ajustar", pois é um fator relevante de cargo para promoção por requisitos
df['formacao'].fillna('Não informado', inplace=True)

In [ ]:
#### Ajustes pontuais: Transformando dados para visualização nos gráficos

In [ ]:
# Mapeamento de gêneros
mapeamento_genero = {
    'Masc': 'Masculino',
    'Fem': 'Feminino',
    'Neutro': 'Neutro',
    'Outros': 'Outros',
    np.nan: 'Não informado'
}

# Aplicando o mapeamento na coluna 'genero' e substituindo NaN por 'Não informado'
df['genero'] = df['genero'].map(mapeamento_genero)

# Mostra o DataFrame com a coluna 'genero' ajustada
display(df.head())

In [ ]:
df.rename(columns={'raca':'etnia'}, inplace = True)

In [ ]:
# Corrigindo a coluna 'etnia' para ter a primeira letra em maiúscula e valores nulos substituídos
df['etnia'] = df['etnia'].fillna('Não informado').apply(lambda x: x.capitalize() if not pd.isnull(x) else x)

# Mostra o DataFrame com a coluna 'etnia' corrigida
display(df.head())

#### Ajustes pontuais: Dados duplicados

In [ ]:
linhas_duplicadas = df.duplicated()

total_linhas_duplicadas = linhas_duplicadas.sum()

if total_linhas_duplicadas > 0:
    print("Existem", total_linhas_duplicadas, "linhas duplicadas no Dataset.")
else:
    print("Não existem linhas duplicadas no Dataset.")

### Análise de dados: Criação da base limpa para desenvolvimento 

#### Análises de verificação: Leitura dos dados importantes

In [ ]:
df_limpo = df.to_csv("base_de_dados_limpa.csv")

In [ ]:
caminho_arquivo_limpo = r'C:\Users\User\Desktop\Ada Hackaton\adahack-2024-dados\Entregas\DD-08\base_de_dados_limpa.csv'
df_limpo = pd.read_csv(caminho_arquivo_limpo)

In [ ]:
valores_distintos = df_limpo['genero'].unique()
print(valores_distintos)

In [ ]:
df_limpo["genero"].value_counts()

In [ ]:
df_limpo["etnia"].value_counts()

In [ ]:
df_limpo[['etnia', 'genero']].value_counts()

In [ ]:
df_limpo[['etnia', 'genero','senioridade']].value_counts()

In [ ]:
#Obs: tem gerente e Gerente
df_limpo['senioridade'] = df_limpo['senioridade'].str.title()
df['senioridade'] = df['senioridade'].str.title()

In [ ]:
df_limpo.dtypes

In [ ]:
df_limpo[['etnia', 'genero', 'senioridade']][(df_limpo['senioridade'] == 'Diretor') | (df_limpo['senioridade'] == 'Gerente')].value_counts()

#### Análises : Cargos x Etnias

In [ ]:
# Definir cargos de alto escalão
cargos_alto_escalao = ['Diretor', 'Gerente']

In [ ]:
# Filtrar pessoas não brancas fora de cargos de alto escalão
nao_brancos_baixo_escalao = df[(df['etnia'] != 'Branca') & (~df['senioridade'].isin(cargos_alto_escalao))]
contagem_nao_brancos_baixo_escalao = nao_brancos_baixo_escalao['senioridade'].value_counts()
contagem_nao_brancos_baixo_escalao

In [ ]:
# Filtrar pessoas não brancas em cargos de alto escalão
nao_brancos_alto_escalao = df[(df['etnia'] != 'Branca') & (df['senioridade'].isin(cargos_alto_escalao))]
contagem_nao_brancos_alto_escalao = nao_brancos_alto_escalao['senioridade'].value_counts()
contagem_nao_brancos_alto_escalao

#### Análises : Etnias x Tempo de Casa

In [ ]:
df_limpo[['etnia', 'tempo_de_casa']].sort_values(by='tempo_de_casa')

In [ ]:
df_limpo[['idade', 'tempo_de_casa']].sort_values(by='tempo_de_casa')

#### Análises : Etnias na corporação

In [ ]:
print(df_limpo['etnia'].value_counts())

#### Análises : Valorização do colaborador por tempo de casa

In [ ]:
#valorização (promoções por tempo de casa)

valores_unicos_formacao = df['formacao'].unique()
valores_unicos_formacao

In [ ]:
#Valorização dos estudos por cargos

contagem_formacao = df.groupby('senioridade')['formacao'].value_counts().unstack()
contagem_formacao.fillna(0, inplace=True)
display(contagem_formacao)

In [ ]:
#Valorização dos estudos por cargos

contagem_tempo = df.groupby('tempo_de_casa')['senioridade'].value_counts().unstack()
contagem_tempo.fillna(0, inplace=True)
display(contagem_tempo)

In [ ]:
df.isnull().sum()

#### Análises : Caso dos estagiários - Lei do estagiário 11788/2008 - Art 11

In [ ]:
#Ponto de atenção: Estagiários com tempo maios de 2 anos. Pela Lei do estagiário isso não é permitido
#Vamos delegar ao RH que faça as devidas trativas: Desligar ou admitir em cargos compativeis com a disponilibidade.
estagiarios_irregulares = df[(df['senioridade'] == 'Estagiario') & (df['tempo_de_casa'] > 2)]

df = df.drop(estagiarios_irregulares.index)
df_menores_irregulares.to_csv('estagiarios_irregulares.csv', index=False)

### Gráficos das análises:

In [ ]:
plt.figure(figsize=(10, 8))
ax = sns.countplot(data=df_limpo, x='etnia', order=df_limpo['etnia'].value_counts().index, palette='viridis')
plt.title('Colaboradores por Etnia')
plt.xlabel('Etnia')
plt.ylabel('Contagem')
plt.xticks(rotation=45, ha='right')

# Adicionar os valores em porcentagem nas colunas do gráfico
total = float(len(df_limpo))
for p in ax.patches:
    ax.annotate('{:.1%}'.format(p.get_height() / total),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 9),
                textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
# Obter a lista de departamentos únicos no DataFrame
departamentos = df_limpo['departamento'].unique()

# Iterar sobre cada departamento e criar um gráfico separado
for departamento in departamentos:
    plt.figure(figsize=(10, 6))
    filtro = df_limpo[df_limpo['departamento'] == departamento]  # Filtrar os dados pelo departamento atual
    ax = sns.countplot(data=filtro, x='formacao',palette='viridis')
    plt.title(f'Distribuição de Formação para o Departamento {departamento}')
    plt.xlabel('Formação')
    plt.ylabel('Contagem')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Adicionar os valores nas colunas do gráfico
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.0f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', 
                    xytext=(0, 9), 
                    textcoords='offset points')

    plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_limpo, x='etnia', y='idade', hue='departamento',palette='viridis')
plt.title('Boxplot de Idade por Etnia e Departamento')
plt.xlabel('Etnia')
plt.ylabel('Idade (anos)')
plt.legend(title='Departamento', loc='upper right')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(data=df_limpo, x='etnia', y='senioridade', hue='departamento',palette='viridis')
plt.title('Boxplot de Senioridade por Etnia e Departamento')
plt.xlabel('Etnia')
plt.ylabel('Senioridade')
plt.legend(title='Departamento', loc='lower right')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(data=df_limpo, x='tempo_de_casa', y='senioridade', hue='departamento',palette='viridis')
plt.title('Boxplot de Senioridade por Tempo de casa e Cargos')
plt.xlabel('Tempo de casa(Anos)')
plt.ylabel('Senioridade')
plt.legend(title='Departamento', loc='lower right')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
ax = sns.countplot(data=df_limpo, x='departamento', hue='genero',palette='viridis')
plt.title('Distribuição de Gênero por Departamento')
plt.xlabel('Departamento')
plt.ylabel('Contagem')
plt.legend(title='Gênero', loc='lower right')
plt.xticks(rotation=45, ha='right')

# Adicionar os valores nas colunas do gráfico
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                xytext=(0, 9), 
                textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
# Lista dos departamentos únicos
departamentos = df_limpo['departamento'].unique()

# Iterar sobre cada departamento e plotar o histograma
for departamento in departamentos:
    plt.figure(figsize=(10, 6))
    filtro = df_limpo[df_limpo['departamento'] == departamento]  # Filtrar dados pelo departamento atual
    sns.histplot(data=filtro, x='idade',hue='departamento',palette='viridis')
    plt.title(f'Histograma de Idade para o Departamento {departamento}')
    plt.xlabel('Idade (anos)')
    plt.ylabel('Contagem')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Adicionar os valores nas colunas do gráfico
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.0f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', 
                    xytext=(0, 9), 
                    textcoords='offset points')

    plt.show()

In [ ]:
# Iterar sobre cada departamento e plotar o histograma
for departamento in departamentos:
    plt.figure(figsize=(10, 6))
    filtro = df_limpo[df_limpo['departamento'] == departamento]  # Filtrar dados pelo departamento atual
    sns.histplot(data=filtro, x='tempo_de_casa',hue='departamento',palette='viridis')
    plt.title(f'Histograma de Tempo de casa (anos) para o Departamento {departamento}')
    plt.xlabel('Tempo de casa (anos)')
    plt.ylabel('Contagem')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Adicionar os valores nas colunas do gráfico
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.0f'), 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', 
                    xytext=(0, 9), 
                    textcoords='offset points')

    plt.show()

In [ ]:
tempos_de_casa = df_limpo['tempo_de_casa'].unique()

In [ ]:
for tempo_de_casa in tempos_de_casa:
    plt.figure(figsize=(10, 6))
    filtro = df_limpo[df_limpo['tempo_de_casa'] == tempo_de_casa]  # Filtrar dados pelo tempo_de_casa


    if len(filtro) > 0:
        sns.histplot(data=filtro, x='idade', hue='tempo_de_casa', palette='viridis')
        plt.title(f'Histograma de Idade (anos) para o Tempo de casa {tempo_de_casa}')
        plt.xlabel('Idade (anos)')
        plt.ylabel('Contagem')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Adicionar os valores nas colunas do gráfico
        for p in ax.patches:
            ax.annotate(format(p.get_height(), '.0f'), 
                        (p.get_x() + p.get_width() / 2., p.get_height()), 
                        ha='center', va='center', 
                        xytext=(0, 9), 
                        textcoords='offset points')

        plt.show()
    else:
        print(f'Sem dados para o tempo de casa {tempo_de_casa}.')

In [ ]:
genero_counts = df_limpo['genero'].value_counts()

In [ ]:
plt.figure(figsize=(7, 5))
genero_counts.plot(kind='bar', cmap='viridis')
plt.title('Distribuição Demográfica por Gênero')
plt.xlabel('Gênero')
plt.ylabel('Número de Colaboradores')
plt.xticks(rotation=0)
plt.show()

In [ ]:
etnia_counts = df_limpo['etnia'].value_counts()

In [ ]:
plt.figure(figsize=(7, 5))
etnia_counts.plot(kind='bar', cmap='viridis')
plt.title('Distribuição Demográfica por Etnia')
plt.xlabel('Etnia')
plt.ylabel('Número de Colaboradores')
plt.xticks(rotation=45)
plt.show()

In [ ]:
etnia_genero_counts = df_limpo.groupby(['etnia', 'genero']).size().unstack(fill_value=0)

In [ ]:
plt.figure(figsize=(7, 5))
etnia_genero_counts.plot(kind='bar', cmap='viridis')
plt.title('Distribuição de Colaboradores por Etnia e Gênero', fontsize=14)
plt.xlabel('Etnia', fontsize=12)
plt.ylabel('Número de Colaboradores', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Gênero', fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
filtro_cargos = (df['senioridade'] == 'Diretor') | (df['senioridade'] == 'Gerente')
dados_filtrados = df[filtro_cargos]

In [ ]:
etnia_sen_counts = dados_filtrados.groupby(['etnia', 'senioridade']).size().unstack(fill_value=0)

In [ ]:
for senioridade in etnia_sen_counts.columns:
    plt.figure(figsize=(8, 6))
    etnia_sen_counts[senioridade].plot(kind='pie', autopct='%1.1f%%', cmap='viridis')
    plt.title(f'Distribuição de Colaboradores por Etnia ({senioridade})', fontsize=14)
    plt.ylabel('')
    plt.legend(title='Senioridade', fontsize=10)
    plt.tight_layout()
    plt.show()

In [ ]:
plt.figure(figsize=(7, 5))
df_limpo['idade'].plot(kind='hist', bins=20, cmap='viridis', edgecolor='black')
plt.title('Distribuição de Idade')
plt.xlabel('Idade')
plt.ylabel('Frequência')

In [ ]:
plt.figure(figsize=(7, 7))
df_limpo['estado'].value_counts().plot(kind='bar', cmap='viridis')
plt.title('Distribuição de Estado')
plt.xlabel('Estado ')
plt.ylabel('Contagem')

plt.tight_layout()
plt.show()

In [ ]:
df['estado'].value_counts

In [ ]:
cargos_lideranca = df_limpo[(df_limpo['senioridade'] == 'Diretor') | (df_limpo['senioridade'] == 'Gerente')]

gerentes = cargos_lideranca[cargos_lideranca['senioridade'] == 'Gerente']
diretores = cargos_lideranca[cargos_lideranca['senioridade'] == 'Diretor']

genero_gerentes = gerentes['genero'].value_counts(normalize=True)
genero_diretores = diretores['genero'].value_counts(normalize=True)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
genero_gerentes.plot(kind='pie', autopct='%1.1f%%', cmap='viridis')
plt.title('Distribuição por Gênero entre Gerentes')
plt.ylabel('')

plt.subplot(1, 2, 2)
genero_diretores.plot(kind='pie', autopct='%1.1f%%', cmap='viridis')
plt.title('Distribuição por Gênero entre Diretores')
plt.ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
df_knn = df_limpo.dropna(subset=['idade', 'tempo_de_casa'])

In [ ]:
# Normalização dos dados
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_knn[['idade', 'tempo_de_casa']])

In [ ]:
# Aplicação do K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
df_knn['cluster'] = kmeans.fit_predict(df_scaled)

In [ ]:
# Plotagem dos clusters
plt.figure(figsize=(10, 6))
plt.scatter(df_scaled[:, 0], df_scaled[:, 1], c=df_knn['cluster'], cmap='viridis', marker='o')
plt.title('Visualização dos Clusters')
plt.xlabel('Idade (normalizada)')
plt.ylabel('Tempo de Casa (normalizado)')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
# Normalização dos dados
scaler = StandardScaler()
df_knn_scaled = scaler.fit_transform(df_knn[['idade', 'tempo_de_casa']])

In [ ]:
# Aplicação do K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
df_knn['cluster'] = kmeans.fit_predict(df_knn_scaled)

# Invertendo a normalização para os valores originais
df_knn['idade_original'] = scaler.inverse_transform(df_knn_scaled)[:, 0]
df_knn['tempo_de_casa_original'] = scaler.inverse_transform(df_knn_scaled)[:, 1]

# Plotagem dos clusters com valores reais
plt.figure(figsize=(10, 6))
plt.scatter(df_knn['idade_original'], df_knn['tempo_de_casa_original'], c=df_knn['cluster'], cmap='viridis', marker='o')
plt.title('Visualização dos Clusters com Valores Reais')
plt.xlabel('Idade')
plt.ylabel('Tempo de Casa')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
#mudar para mapa

counts = df.groupby(['etnia', 'formacao']).size().unstack(fill_value=0)


plt.figure(figsize=(10, 8))
sns.heatmap(counts, cmap='viridis', annot=True, fmt='d')
plt.xlabel('Formação', fontsize=12)
plt.ylabel('Etnia', fontsize=12)
plt.title('Relação entre Etnia e Formação', fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
senioridades_desejadas = ['Analista Pleno', 'Analista Junior', 'Analista Senior', 'Estagiario',]
df_limpo_filtrado = df_limpo[df_limpo['senioridade'].isin(senioridades_desejadas)]

dados_agrupados = df_limpo_filtrado.groupby(['senioridade', 'formacao']).size().unstack(fill_value=0)

dados_agrupados.plot(kind='bar', stacked=False, figsize=(10, 8), cmap='viridis')
plt.title('Número de Colaboradores por Senioridade e Formação')
plt.xlabel('Senioridade')
plt.ylabel('Número de Colaboradores')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Formação')
plt.show()

In [ ]:
coordenadas_estados = {
    'AC': (-9.97499, -67.8243),
    'AL': (-9.66599, -35.735),
    'AP': (0.0390458, -51.0664),
    'AM': (-3.11903, -60.0212),
    'BA': (-12.9714, -38.5014),
    'CE': (-3.71722, -38.5433),
    'DF' : (-15.7795, -47.9297),
    'ES': (-20.3155, -40.3128),
    'GO': (-16.6864, -49.2643),
    'MA': (-2.53911, -44.2829),
    'MT': (-15.601, -56.0974),
    'MS': (-20.4428, -54.6464),
    'MG': (-19.9183, -43.9346),
    'PA': (-1.4554, -48.4898),
    'PB': (-7.11949, -34.8451),
    'PR': (-25.4278, -49.2731),
    'PE': (-8.04756, -34.877),
    'PI': (-5.08921, -42.8016),
    'RJ': (-22.9068, -43.1729),
    'RN': (-5.79448, -35.211),
    'RS': (-30.0346, -51.2177),
    'RO': (-8.76116, -63.9004),
    'RR': (2.82384, -60.6753),
    'SC': (-27.5954, -48.548),
    'SP': (-23.5505, -46.6333),
    'SE': (-10.9472, -37.0731),
    'TO': (-10.1689, -48.3317)
}

def adicionar_coordenadas(df_limpo, coordenadas_estados):
    df_limpo['latitude'] = df_limpo['estado'].map(lambda x: coordenadas_estados[x][0])
    df_limpo['longitude'] = df_limpo['estado'].map(lambda x: coordenadas_estados[x][1])
    return df_limpo


df_limpo = adicionar_coordenadas(df_limpo, coordenadas_estados)

In [ ]:
df_limpo

In [ ]:
# Verificar se a coluna 'Unnamed:' está presente no DataFrame
if 'Unnamed:' in df.columns:
    df = df.drop('Unnamed:', axis=1)

In [ ]:
df_limpo

In [ ]:
df_limpo.dtypes

In [ ]:
app = dash.Dash(__name__)


app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown_formacao',
        options=[{'label': formacao, 'value': formacao} for formacao in df_limpo['formacao'].unique()],
        value=df_limpo['formacao'].unique().tolist(),
        multi=True,
        placeholder='Selecione a Formação'
    ),
    dcc.Dropdown(
        id='dropdown_etnia',
        options=[{'label': etnia, 'value': etnia} for etnia in df_limpo['etnia'].unique()],
        value=df_limpo['etnia'].unique().tolist(),
        multi=True,
        placeholder='Selecione a Etnia'
    ),
    html.Div(id='map_container')
])

@app.callback(
    Output('map_container', 'children'),
    [Input('dropdown_formacao', 'value'),
     Input('dropdown_etnia', 'value')]
)
def update_map(selected_formacoes, selected_etnias):

    filtered_df_limpo = df_limpo[df_limpo['formacao'].isin(selected_formacoes) & df_limpo['etnia'].isin(selected_etnias)]
    
  
    contagem_combinacoes = filtered_df_limpo.groupby(['formacao', 'etnia']).size().reset_index(name='contagem')
    

    contagem_combinacoes = contagem_combinacoes.merge(filtered_df_limpo[['formacao', 'etnia', 'latitude', 'longitude']].drop_duplicates(), on=['formacao', 'etnia'])
    

    mapa = folium.Map(location=[-14.235004, -51.925282], zoom_start=4)
    
  
    for _, row in contagem_combinacoes.iterrows():
        coordenadas = [row['latitude'], row['longitude']]
        peso = row['contagem']
        HeatMap([coordenadas + [peso]], radius=15).add_to(mapa)
    

    map_html = mapa._repr_html_()
    
    return html.Iframe(srcDoc=map_html, width='100%', height='600')


if __name__ == '__main__':
    app.run_server(debug=True)